In [ ]:
import os
import cv2
import tensorflow as tf
import numpy as np
import json

In [ ]:
from utilities import *

In [ ]:
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
#LABEL_MAP_NAME = 'mscoco_label_map.pbtxt'
LABEL_MAP_NAME = 'mscoco_label_map.json'

CWD_PATH = os.getcwd()
MODELS_PATH = os.path.abspath(os.path.join(CWD_PATH, 'models'))
LABELS_PATH = os.path.abspath(os.path.join(CWD_PATH, 'labels'))
DATA_PATH = os.path.abspath(os.path.join(CWD_PATH, 'data'))

VIDEO_FILE = 'videoplayback.mp4'
VIDEO_PATH = os.path.join(DATA_PATH, VIDEO_FILE)

MODEL_PATH = os.path.join(MODELS_PATH, MODEL_NAME, 'frozen_inference_graph.pb')
LABEL_MAP_PATH = os.path.join(LABELS_PATH, LABEL_MAP_NAME)

THRESHOLD = 0

In [ ]:
with open(LABEL_MAP_PATH) as json_file:
    labels = json.load(json_file)

In [ ]:
labels

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(MODEL_PATH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [ ]:
def filter_boxes(min_score, boxes, scores, classes, categories):
    """Return boxes with a confidence >= `min_score`"""
    n = len(classes)
    idxs = []
    
    for i in range(n):
        if scores[i]>=min_score:
            idxs.append(i)
    
    filtered_boxes = boxes[idxs, ...]
    filtered_scores = scores[idxs, ...]
    filtered_classes = classes[idxs, ...]
    return filtered_boxes, filtered_scores, filtered_classes



In [ ]:
 def detect_objects(image_np, sess, detection_graph, threshold=0):
        # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image_np, axis=0)
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

        # Each box represents a part of the image where a particular object was detected.
        boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

        # Each score represent level of confidence for each of the objects.
        scores = detection_graph.get_tensor_by_name('detection_scores:0')
        classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')

        # Actual detection.
        (boxes, scores, classes, num_detections) = sess.run(
            [boxes, scores, classes, num_detections],
            feed_dict={image_tensor: image_np_expanded})
        
        boxes, scores, classes, = filter_boxes(0.5, boxes[0], scores[0], classes[0], 1)

        return (boxes, scores, classes, num_detections)

In [ ]:
# WEBCAM
video_stream = VideoStream(src=1).start() 
# VIDEO File
# video_stream = VideoStream(src=VIDEO_PATH).start()
sess = tf.Session(graph=detection_graph) 
    
while video_stream.grabbed:
    frame, counter = video_stream.read()
    
    # do some zoo model detection on the frame
    data = detect_objects(frame, sess, detection_graph)
    
    height = frame.shape[0]
    width = frame.shape[1]
    
    count = 0 
    for item in data[0]:
        obj = str(int(data[2][count]))

        cv2.putText(
            frame,
            labels[obj]["name"] + " - " + str(data[1][count]),
            (int(width * item[1]), int(height * item[0])),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (255, 0, 0),
            1,
            cv2.LINE_AA
        )

        cv2.rectangle(
            frame,
            (int(width * item[1]), int(height * item[0])), 
            (int(width * item[3]), int(height * item[2])), 
            (0, 255, 0),
            1
        )
        count = count + 1
        
    
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_stream.stop()
cv2.destroyAllWindows()
cv2.waitKey(1000) # to autoclose window after a few seconds